### Test collectory update of isPrivate flag

In [1]:
import json
from pathlib import Path
import pandas as pd
import requests

cwd = Path.cwd()
colfile = cwd / 'data/lists.test-collectory-metadata.csv'
# functions

def get_collectory_metadata(druid, collectory_url, headers):
    metadata = ""
    metadata_df = pd.DataFrame()
    response = requests.get(collectory_url, headers)
    if response.status_code == 200:
        metadata = response.json()
        print(f"{druid} -  Collectory metadata extracted for: {collectory_url}")
    else:
        print(
            f"{druid} - Collectory metadata request for failed with status code {response.status_code}, {collectory_url}"
        )
    metadata_df = pd.DataFrame([metadata])
    return metadata_df


def update_collectory_dr(row, collectory_url, authorization) -> str:
    """
    Update list - set isPrivate flag in collectory DR

    :param row: list information from dataframe
    :return str: Update request response code
    """

    # drID = row['dataResourceUid']
    jstr = row.to_dict()
    print("POST to %s", collectory_url)
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": authorization
    }
    try:
        with requests.post(collectory_url, json=jstr, headers=headers) as response:
            if response.status_code == 200 or response.status_code == 201:
                return str(response.status_code)
            else:
                response.raise_for_status()
    except Exception as e:
        print("Error in creating %s: %s for %s", collectory_url, jstr, e)
        response.raise_for_status()


In [12]:
# Set dataresource to retrieve
druid = 'dr21322'
druid = 'dr22810'

# Set collectory base URL
# collectory_url = "https://collections-test.ala.org.au/ws/dataResource/" + druid
collectory_url = "https://collections.test.ala.org.au/ws/dataResource/" + druid

# Collectory  api-key
authorization_api = "dba60c96-380f-4793-90c8-5dbc4764ca4b"    

# Collectory JWT Token
JWTToken = 'eyJraWQiOiI2UEpOaFwvdU5EYlBIWlk4Y2xmTHJvMnBKUnJhTFRXTnpaU0tOcVdka3Y0az0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI1MDJkYmE3Yy00YWFjLTQ2ZWMtOGY4Ni0xM2JkZGMxNzgyYjYiLCJjb2duaXRvOmdyb3VwcyI6WyJjb2xsZWN0aW9uX2FkbWluIiwidXNlciIsImNvbGxlY3RvcnNfYWRtaW4iLCJhZG1pbiIsImRhdGFfcHVibGlzaGVyIl0sImlzcyI6Imh0dHBzOlwvXC9jb2duaXRvLWlkcC5hcC1zb3V0aGVhc3QtMi5hbWF6b25hd3MuY29tXC9hcC1zb3V0aGVhc3QtMl9PT1hVOUdXMzkiLCJ2ZXJzaW9uIjoyLCJjbGllbnRfaWQiOiI0NjFodDJjOHBxdnVzMGVyNzNmcDBkMWlrMiIsIm9yaWdpbl9qdGkiOiJkMzdkNTZlMy01Mjc1LTQxM2EtYTI2Ny00NzNmNjQyNDdhYTIiLCJ0b2tlbl91c2UiOiJhY2Nlc3MiLCJzY29wZSI6ImFsYVwvcm9sZXMgb3BlbmlkIHByb2ZpbGUgZW1haWwiLCJhdXRoX3RpbWUiOjE3NjExOTgxMDYsImV4cCI6MTc2MTI4NDUwNiwiaWF0IjoxNzYxMTk4MTA2LCJqdGkiOiI1OWZiNTcxNy04YmMxLTRlNDQtOTgxNi0yNjk1ODhjOWE2ZjciLCJ1c2VybmFtZSI6IjU2NTkyIn0.v9yNSV3_XXgOTPf1vVfXnXmcDEkqTp4p6Dz4sbFkUPUiW_B5DfPAly14_ab7cAH9Cma2sSbCzZuQ2J-mvecdZXJTwBvXEcqoZS_phPfkF2MLA6_9stjlUFR0bmjFtcDLJ1MPZcz79GQK5UVAsLrUBU16a091dFJFkLX4_KCbLtNTRah5pTzbP9skEcWaB53e94TzZmUn4U-Lpa7tRx2h2PwlFDSFBPHBTg2LYCYh7THAZFUZS7xKqx8UqbIgH0JdpRFlf15PN75YBP0EltnKhh6v6JBOZ5dj6aW9uug-2n2PKsZ__EA_KzNoJ8sIQDy3BAmOHdL6VXlmdHxvMcfFMg'
authorization_jwt = f"Bearer {JWTToken}"

header_noauth = {
    "Content-Type": "application/json",
    "Accept": "application/json",
}
meta_df = get_collectory_metadata(druid, collectory_url, header_noauth)
# Modify metadata
# meta_df['isPrivate'] = 'True'
meta_df

dr22810 -  Collectory metadata extracted for: https://collections.test.ala.org.au/ws/dataResource/dr22810


,name,acronym,uid,guid,address,phone,email,pubShortDescription,pubDescription,techDescription,...,publicArchiveUrl,gbifArchiveUrl,downloadLimit,gbifDataset,isShareableWithGBIF,verified,gbifRegistryKey,doi,repatriationCountry,createdByID
0,00 - dr22810 Rose Test - DR-List,None,dr22810,None,None,None,None,None,,This list was first uploaded by Rosemary O'Con...,...,https://biocache.ala.org.au/archives/gbif/dr22...,https://biocache.ala.org.au/archives/gbif/dr22...,0,False,True,False,None,None,None,None


In [ ]:
# Update collectory metadata

# Update using api-key
# meta_df['name'] = 'api-' + meta_df['name']
# meta_df['cUpdStatus-api'] = meta_df.apply(lambda row: update_collectory_dr(row, collectory_url, authorization_api), axis=1)
# meta_df

In [14]:
# Update using jwt
meta_df['name'] = meta_df['name'] + "-jwt"
meta_df['cUpdStatus-jwt'] = meta_df.apply(lambda row: update_collectory_dr(row, collectory_url, authorization_jwt), axis=1)
meta_df

POST to %s https://collections.test.ala.org.au/ws/dataResource/dr22810


,name,acronym,uid,guid,address,phone,email,pubShortDescription,pubDescription,techDescription,...,gbifArchiveUrl,downloadLimit,gbifDataset,isShareableWithGBIF,verified,gbifRegistryKey,doi,repatriationCountry,createdByID,cUpdStatus-jwt
0,api-00 - dr22810 Rose Test - DR-List-jwt,None,dr22810,None,None,None,None,None,,This list was first uploaded by Rosemary O'Con...,...,https://biocache.ala.org.au/archives/gbif/dr22...,0,False,True,False,None,None,None,None,200
